# XGBoost Server Test with Iris Dataset

In [126]:
import os

import xgboost as xgb
from sklearn.datasets import load_iris

In [127]:
model_dir = "."

BOOSTER_FILE = "model.json"
BOOSTER_FILE_DEPRECATED = "model.bst"

In [128]:
iris = load_iris()
y = iris["target"]
X = iris["data"]
dtrain = xgb.DMatrix(X, label=y)
param = {
    "max_depth": 6,
    "eta": 0.1,
    "nthread": 4,
    "num_class": 10,
    "objective": "multi:softmax",
}
xgb_model = xgb.train(params=param, dtrain=dtrain)

model_file_json = os.path.join((model_dir), BOOSTER_FILE)
model_file_bst= os.path.join((model_dir), BOOSTER_FILE_DEPRECATED)
xgb_model.save_model(model_file_json)
xgb_model.save_model(model_file_bst)

## Wrap model using s2i

In [129]:
!cd .. && make docker-build

s2i build \
	-E environment \
	./xgboostserver \
	seldonio/seldon-core-s2i-python312:1.19.0-dev \
	seldonio/xgboostserver:1.19.0-dev
Before assembling
---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 31.1 MB/s  0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 MB 43.3 MB/s  0:00:05
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 56.4 MB/s  0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 56.2 MB/s  0:00:05


created path: ./licenses/license_info.csv
created path: ./licenses/license.txt
After successful assembling
Build completed successfully


In [130]:
!docker rm xgboostserver --force

Error response from daemon: No such container: xgboostserver


In [131]:
!docker run --rm -d --name "xgboostserver" -p 9000:9000 -p 5000:5000 -e PREDICTIVE_UNIT_PARAMETERS='[{"type":"STRING","name":"model_uri","value":"file:///model"}]' -v ${PWD}:/model seldonio/xgboostserver:1.19.0-dev

df4fb3554aa0082cbe12cdeb25d834bd4e0384541c0c90c58dab216e6c400788


## REST test

Send some random features that conform to the contract

In [132]:
!seldon-core-tester contract.json 0.0.0.0 9000 -p

----------------------------------------
SENDING NEW REQUEST:

[[6.695 2.275 9.434 2.   ]]
RECEIVED RESPONSE:
meta {
}
data {
  ndarray {
    values {
      number_value: 2
    }
  }
}




## grpc test

Test using NDArray payload

In [133]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc

----------------------------------------
SENDING NEW REQUEST:

[[7.014 2.032 1.841 0.374]]
RECEIVED RESPONSE:
meta {
}
data {
  ndarray {
    values {
      number_value: 0
    }
  }
}




Test using Tensor payload

In [134]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc --tensor

----------------------------------------
SENDING NEW REQUEST:

[[6.172 4.71  7.057 0.701]]
RECEIVED RESPONSE:
meta {
}
data {
  tensor {
    shape: 1
    values: 2
  }
}




In [135]:
!pytest -s .

============================= test session starts ==============================
platform linux -- Python 3.12.0, pytest-9.0.2, pluggy-1.6.0
rootdir: /home/viktortaskov/forked-seldon-projects/seldon-core
configfile: pyproject.toml
plugins: anyio-4.12.0
collected 4 items                                                              

.Fput.py 
.Fedict.py 

=================================== FAILURES ===================================
__________________________ test_predict_invalid_input __________________________

model_uri = '/tmp/tmpu1pvo7v5'

    def test_predict_invalid_input(model_uri):
        # Create an instance of XGBoostServer with the model URI
        server = XGBoostServer(model_uri)
        server.load()
    
        X_test = np.random.rand(10, 3)  # Incorrect number of features
>       with pytest.raises(ValueError):
E       Failed: DID NOT RAISE <class 'ValueError'>

test_metadata_and_input.py:43: Failed
__________________________ test_predict_invalid_input ____________

In [118]:
!docker rm xgboostserver --force

xgboostserver
